<a href="https://colab.research.google.com/github/Alihassan7726/BERT-/blob/main/Text_classification_using_BERT_by_%F0%9F%A4%97_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=right> Fine Tuning BERT for text classification by using HuggingFace library 🤗

## Table of contents :
- **[Step 1]()**: Importing packages and downloading data  
- **[Step 2]()**: Basic Cleansing of Data
- **[Step 3]()**: Splitting into train and val sets
- **[Step 4]()**: Installing transformers and datasets libraries
- **[Step 5]()**: Downloading BERT tokenizer and tokenizing inputs as per BERT requirements .
- **[Step 6]()**: Creating input data pipeline using `tf.data` API
- **[Step 7]()**: Defining model parameters and training our BERT model using `TFTrainingArguments` and `TFTrainer` constructors .
- **[Step 8]()**: Evaluating our BERT model


## Step 1: Importing packages and downloading data . 


In [ ]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.width',1000)
pd.set_option('max_colwidth', 1000) 
import pandas as pd

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stpwrds = set(stopwords.words('english'))
stpwrds.discard('not')
stpwrds.discard('no')
import os
import re
from nltk.stem import WordNetLemmatizer as wnl
import sklearn


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Data is fetched and stored in tmp folder with name bbc-text.csv
!wget --no-check-certificate \
    https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2021-06-17 15:14:01--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.03s   

2021-06-17 15:14:02 (157 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [ ]:
df = pd.read_csv('/tmp/bbc-text.csv' , delimiter = ',')
df.head()

,category,text
0,tech,tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices. one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes like the us s tivo and the uk s sky+ system allow people to record store play pause and forward wind tv programmes when they want. essentially the technology allows for much more personalised tv. they are also being built-in to h...
1,business,worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn (£5.8bn) fraud never made accounting decisions a witness has told jurors. david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors. on monday defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination he asked mr myers if he ever knew mr ebbers make an accounting decision . not that i am aware of mr myers replied. did you ever know mr ebbers to make an accounting entry into worldcom books mr weingarten pressed. no replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief fina...
2,sport,tigers wary of farrell gamble leicester say they will not be rushed into making a bid for andy farrell should the great britain rugby league captain decide to switch codes. we and anybody else involved in the process are still some way away from going to the next stage tigers boss john wells told bbc radio leicester. at the moment there are still a lot of unknowns about andy farrell not least his medical situation. whoever does take him on is going to take a big big gamble. farrell who has had persistent knee problems had an operation on his knee five weeks ago and is expected to be out for another three months. leicester and saracens are believed to head the list of rugby union clubs interested in signing farrell if he decides to move to the 15-man game. if he does move across to union wells believes he would better off playing in the backs at least initially. i m sure he could make the step between league and union by being involved in the centre said wells. ...
3,sport,yeading face newcastle in fa cup premiership side newcastle united face a trip to ryman premier league leaders yeading in the fa cup third round. the game - arguably the highlight of the draw - is a potential money-spinner for non-league yeading who beat slough in the second round. conference side exeter city who knocked out doncaster on saturday will travel to old trafford to meet holders manchester united in january. arsenal were drawn at home to stoke and chelsea will play host to scunthorpe. the only other non-league side in the draw are hinckley united who held brentford to a goalless draw on sunday. they will meet league one leaders luton if they win their replay against martin allen s team at griffin park. a number of premiership teams face difficult away games against championship sides on the weekend of 8/9 january. third-placed everton visit plymouth liverpool travel to burnley crystal palace go to sunderland fulham face carling cup semi-finalists watford bolton...
4,entertainment,ocea

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
df['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [ ]:
# For simplicity we'll be taking only 2 classes for this task
df = df[df['category'].isin(['tech','business'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911 entries, 0 to 2220
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  911 non-null    object
 1   text      911 non-null    object
dtypes: object(2)
memory usage: 21.4+ KB


## Step 2. Basic Cleansing of Data

In [ ]:
# A simple function for data cleaning

def clean_text(text):  
    lower = text.lower()
    words = re.sub(r"(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", lower )  
    #pattren = r"[^\w\s\d]"
    #words = re.sub(pattren, "", lower)
    words2 = words.split()
    #print(words2)
    final_words =  [wnl().lemmatize(word , pos = 'v') for word in words2 if word not in stopwords.words('english')]
    final_words = ' '.join(final_words)
    return(final_words)
    
df['Cleaned-text'] = df['text'].apply(clean_text)
df.head()

,category,text,Cleaned-text
0,tech,tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices. one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes like the us s tivo and the uk s sky+ system allow people to record store play pause and forward wind tv programmes when they want. essentially the technology allows for much more personalised tv. they are also being built-in to h...,tv future hand viewers home theatre systems plasma highdefinition tvs digital video recorders move live room way people watch tv radically different five years time accord expert panel gather annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us lead trend program content deliver viewers via home network cable satellite telecoms company broadband service providers front room portable devices one talkedabout technologies ces digital personal video recorders dvr pvr settop box like us tivo uk sky system allow people record store play pause forward wind tv program want essentially technology allow much personalise tv also builtin highdefinition tv set big business japan us slower take europe lack highdefinition program people forward wind advert also forget abide network channel schedule put together alacarte entertainment us network cable satellite company worry mean term advertise revenues well brand identity viewer loyalty channel alth...
1,business,worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn (£5.8bn) fraud never made accounting decisions a witness has told jurors. david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors. on monday defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination he asked mr myers if he ever knew mr ebbers make an accounting decision . not that i am aware of mr myers replied. did you ever know mr ebbers to make an accounting entry into worldcom books mr weingarten pressed. no replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief fina...,worldcom boss leave book alone former worldcom boss bernie ebbers accuse oversee bn bn fraud never make account decisions witness tell jurors david myers make comment question defence lawyers argue mr ebbers responsible worldcom problems phone company collapse prosecutors claim losses hide protect firm share mr myers already plead guilty fraud assist prosecutors monday defence lawyer reid weingarten try distance client allegations cross examination ask mr myers ever know mr ebbers make account decision aware mr myers reply ever know mr ebbers make account entry worldcom book mr weingarten press reply witness mr myers admit order false account entries request former worldcom chief financial officer scott sullivan defence lawyers try paint mr sullivan admit fraud testify later trial mastermind behind worldcom account house card mr ebbers team meanwhile look portray affable boss admission pe graduate economist whatever abilities mr ebbers transform worldcom relative unknown bn tel

In [ ]:
df.drop(columns=['text'],inplace=True)
df.head(2)

,category,Cleaned-text
0,tech,tv future hand viewers home theatre systems plasma highdefinition tvs digital video recorders move live room way people watch tv radically different five years time accord expert panel gather annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us lead trend program content deliver viewers via home network cable satellite telecoms company broadband service providers front room portable devices one talkedabout technologies ces digital personal video recorders dvr pvr settop box like us tivo uk sky system allow people record store play pause forward wind tv program want essentially technology allow much personalise tv also builtin highdefinition tv set big business japan us slower take europe lack highdefinition program people forward wind advert also forget abide network channel schedule put together alacarte entertainment us network cable satellite company worry mean term advertise revenues well brand identity viewer loyalty channel alth...
1,business,worldcom boss leave book alone former worldcom boss bernie ebbers accuse oversee bn bn fraud never make account decisions witness tell jurors david myers make comment question defence lawyers argue mr ebbers responsible worldcom problems phone company collapse prosecutors claim losses hide protect firm share mr myers already plead guilty fraud assist prosecutors monday defence lawyer reid weingarten try distance client allegations cross examination ask mr myers ever know mr ebbers make account decision aware mr myers reply ever know mr ebbers make account entry worldcom book mr weingarten press reply witness mr myers admit order false account entries request former worldcom chief financial officer scott sullivan defence lawyers try paint mr sullivan admit fraud testify later trial mastermind behind worldcom account house card mr ebbers team meanwhile look portray affable boss admission pe graduate economist whatever abilities mr ebbers transform worldcom relative unknown bn telecom...


## Step 3. Splitting into train and val sets

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test , y_train , y_test  = train_test_split(df['Cleaned-text'], df['category'] , stratify = df['category'], test_size = 0.1, random_state = 12)

print(X_train.shape)
print(X_test.shape)

(819,)
(92,)


In [ ]:
#del(df)

In [ ]:
X_train = list(X_train)
X_test = list(X_test)
y_train = list(y_train)
y_test = list(y_test)

In [ ]:
y_train = list(pd.get_dummies(y_train , drop_first=True)['tech'])
y_test = list(pd.get_dummies(y_test , drop_first=True)['tech'])
print(" Train shape : ",len(y_train))
print(" Test shape : ",len(y_test))

 Train shape :  819
 Test shape :  92


## Step 4. Installing transformers and datasets libraries

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 25.0MB/s 
     |████████████████████████████████| 901kB 35.1MB/s 


In [ ]:
pip install datasets


     |████████████████████████████████| 245kB 4.3MB/s 
     |████████████████████████████████| 245kB 28.8MB/s 
     |████████████████████████████████| 122kB 27.2MB/s 


In [ ]:
## Checking weather Datasets is correctly installed
from datasets import load_dataset 
print(load_dataset('squad', split='train')[0])

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16. Subsequent calls will reuse this data.
{'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '57

## Step 5. Downloading BERT tokenizer and tokenizing inputs as per BERT requirements . 

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
## Encoding Data
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

## Step 6. Creating input data pipeline using `tf.data` API

In [ ]:
## Tensor DataSet 
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

## Step 7. Defining model parameters and training our BERT model using `TFTrainingArguments` and `TFTrainer` constructors .   

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=compute_metrics             
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

## Step 8. Evaluating our BERT model

In [ ]:
trainer.evaluate(test_dataset)

{'eval_accuracy': 1.0, 'eval_loss': 0.005989821627736092}

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-3.2508843 ,  3.212054  ],
       [ 3.3492355 , -4.001416  ],
       [ 3.340288  , -4.003615  ],
       [ 3.2400796 , -3.8998575 ],
       [-3.2137551 ,  3.1746154 ],
       [ 3.3109818 , -3.9492047 ],
       [-3.2562022 ,  3.2215192 ],
       [-3.266472  ,  3.235261  ],
       [-3.265515  ,  3.2363207 ],
       [-3.202193  ,  3.1625144 ],
       [-3.2609427 ,  3.2268364 ],
       [ 3.3044627 , -3.9916055 ],
       [-3.2563384 ,  3.2297702 ],
       [ 3.3482242 , -4.0100346 ],
       [-3.2523575 ,  3.2170222 ],
       [-3.259504  ,  3.2384155 ],
       [ 3.3055372 , -3.9993632 ],
       [ 3.3356042 , -4.0099053 ],
       [-3.2579834 ,  3.2283852 ],
       [-3.2493353 ,  3.2256467 ],
       [ 3.3199775 , -4.0135355 ],
       [-3.246587  ,  3.2071621 ],
       [ 3.3382592 , -3.9814615 ],
       [ 3.3117468 , -3.930558  ],
       [ 3.3615909 , -3.9991746 ],
       [-3.258814  ,  3.227305  ],
       [ 3.3379686 , -4.0271997 ],
       [ 3.3285537 , -3.96

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
output

array([1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1], dtype=int32)

In [ ]:
## Confusion matrix
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[52,  0],
       [ 0, 40]])

In [ ]:
## Test Accuracy
metric.compute(predictions=output, references=y_test)

{'accuracy': 1.0}